In [2]:
# Importing Libraries:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import findspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.sql.functions import *
import findspark
import pyspark.sql.functions as F

from pyspark.sql.functions import concat_ws, col
findspark.init()

In [3]:
# Reading the data
data = pd.read_csv('JW_Marriott.csv', encoding= 'unicode_escape') 
data=data.drop(['Unnamed: 0'], axis=1)
data

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,assigned_room_type,booking_changes,deposit_type,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,duration_of_stay
0,JW Marriott Hotel,0,6,2018,July,27,1,0,2,1,...,A,0,No Deposit,6.0,0,Transient,0.00,0,0,2
1,JW Marriott Hotel,1,88,2018,July,27,1,0,4,2,...,A,0,No Deposit,9.0,0,Transient,76.50,0,1,4
2,JW Marriott Hotel,1,65,2018,July,27,1,0,4,1,...,A,0,No Deposit,9.0,0,Transient,68.00,0,1,4
3,JW Marriott Hotel,1,92,2018,July,27,1,2,4,2,...,A,0,No Deposit,9.0,0,Transient,76.50,0,2,6
4,JW Marriott Hotel,1,100,2018,July,27,2,0,2,2,...,A,0,No Deposit,9.0,0,Transient,76.50,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47673,JW Marriott Hotel,0,23,2017,August,35,30,2,5,2,...,A,0,No Deposit,394.0,0,Transient,96.14,0,0,7
47674,JW Marriott Hotel,0,102,2017,August,35,31,2,5,3,...,E,0,No Deposit,9.0,0,Transient,225.43,0,2,7
47675,JW Marriott Hotel,0,34,2017,August,35,31,2,5,2,...,D,0,No Deposit,9.0,0,Transient,157.71,0,4,7
47676,JW Marriott Hotel,0,109,2017,August,35,31,2,5,2,...,A,0,No Deposit,89.0,0,Transient,104.40,0,0,7


In [4]:
# Number of rows and columns in the data set
rows,cols=data.shape

In [5]:
# Names of all the column
for col in data.columns:
    print(col)

hotel
is_canceled
lead_time
arrival_date_year
arrival_date_month
arrival_date_week_number
arrival_date_day_of_month
stays_in_weekend_nights
stays_in_week_nights
adults
children
babies
meal
country
distribution_channel
is_repeated_guest
previous_cancellations
previous_bookings_not_canceled
reserved_room_type
assigned_room_type
booking_changes
deposit_type
agent
days_in_waiting_list
customer_type
adr
required_car_parking_spaces
total_of_special_requests
duration_of_stay


In [6]:
#Total number of duplicated rows--> no duplicated row 
data.duplicated().sum()

2

In [7]:
# Dropping duplicated rows
data=data.drop_duplicates()

In [8]:
# Checking if there are missing data
data.isnull().values.any()

False

In [9]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

23/06/09 14:07:13 WARN Utils: Your hostname, Lojins-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.12 instead (on interface en0)
23/06/09 14:07:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/09 14:07:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
# Loading the data into spark dataframe
df = spark.createDataFrame(data) 
# Checking that columns were inserted correctly 
df.printSchema()

root
 |-- hotel: string (nullable = true)
 |-- is_canceled: long (nullable = true)
 |-- lead_time: long (nullable = true)
 |-- arrival_date_year: long (nullable = true)
 |-- arrival_date_month: string (nullable = true)
 |-- arrival_date_week_number: long (nullable = true)
 |-- arrival_date_day_of_month: long (nullable = true)
 |-- stays_in_weekend_nights: long (nullable = true)
 |-- stays_in_week_nights: long (nullable = true)
 |-- adults: long (nullable = true)
 |-- children: double (nullable = true)
 |-- babies: long (nullable = true)
 |-- meal: string (nullable = true)
 |-- country: string (nullable = true)
 |-- distribution_channel: string (nullable = true)
 |-- is_repeated_guest: long (nullable = true)
 |-- previous_cancellations: long (nullable = true)
 |-- previous_bookings_not_canceled: long (nullable = true)
 |-- reserved_room_type: string (nullable = true)
 |-- assigned_room_type: string (nullable = true)
 |-- booking_changes: long (nullable = true)
 |-- deposit_type: string 

Promotion One

In [19]:
from pyspark.sql.functions import col

# We assume that families are at least one children or one baby with at least one adult
query_df2 = df.filter((col('adults') >= 1) &  (col('is_canceled') == 1) & ((col('children') >= 1) | (col('babies') >= 1)))
query_df2.show()

#  Getting adults who did  cancel their reservations
query_df3 = df.filter((col('adults') >= 1) &  (col('is_canceled') == 1) & ((col('children') == 0) & (col('babies') == 0)))
query_df3.show()


print("The count of family who did cancel their reservation",query_df2.count())

print("The count of adults who did cancel their reservation",query_df3.count())

+-----------------+-----------+---------+-----------------+------------------+------------------------+-------------------------+-----------------------+--------------------+------+--------+------+----+-------+--------------------+-----------------+----------------------+------------------------------+------------------+------------------+---------------+------------+-----+--------------------+---------------+------+---------------------------+-------------------------+----------------+
|            hotel|is_canceled|lead_time|arrival_date_year|arrival_date_month|arrival_date_week_number|arrival_date_day_of_month|stays_in_weekend_nights|stays_in_week_nights|adults|children|babies|meal|country|distribution_channel|is_repeated_guest|previous_cancellations|previous_bookings_not_canceled|reserved_room_type|assigned_room_type|booking_changes|deposit_type|agent|days_in_waiting_list|  customer_type|   adr|required_car_parking_spaces|total_of_special_requests|duration_of_stay|
+---------------

We find out that adults cancelled there reservations more than families , so we will encourage them by giving them special offer of having both Spa and Wellness Facilities plus a discount of 15% on their package

Promotion Two

In [12]:
# Checking the deposite type effect on cancellation/reservation rate
from pyspark.sql.functions import col

# Getting the count of each deposit type 
query= df.groupby('deposit_type','is_canceled')\
.agg(count("*").alias("count"))\
.sort("deposit_type", ascending=False)

# Changing the 0/1 into categorical values
query = query.withColumn("is_canceled", when(query.is_canceled == "0","Reserved") \
      .when(query.is_canceled == "1","Cancelled") \
      .otherwise(query.is_canceled))

# Adding total count of each type to the query
total_counts = query.groupby('deposit_type').agg(F.sum('count').alias('total_count'))
query = query.join(total_counts, 'deposit_type', 'left')

# Filtering the data to get the reserved data separate from the cancelled
reserved = query.filter((col('is_canceled') == 'Reserved'))
cancelled=query.filter((col('is_canceled') == 'Cancelled'))

# Adding the cancellation rate and reservation rate
cancelled=cancelled.withColumn("Cancellation_Rate",  ((F.col("count"))/ F.col("total_count"))*100 )
reserved=reserved.withColumn("Reservation_Rate",  ((F.col("count"))/ F.col("total_count"))*100 )

cancelled=cancelled.sort("Cancellation_Rate", ascending=False)
reserved=reserved.sort("Reservation_Rate", ascending=False)

reserved.show()
cancelled.show()

+------------+-----------+-----+-----------+------------------+
|deposit_type|is_canceled|count|total_count|  Reservation_Rate|
+------------+-----------+-----+-----------+------------------+
|  No Deposit|   Reserved|32613|      46970| 69.43368107302533|
|  Refundable|   Reserved|    3|         13|23.076923076923077|
|  Non Refund|   Reserved|    3|        693|0.4329004329004329|
+------------+-----------+-----+-----------+------------------+

+------------+-----------+-----+-----------+------------------+
|deposit_type|is_canceled|count|total_count| Cancellation_Rate|
+------------+-----------+-----+-----------+------------------+
|  Non Refund|  Cancelled|  690|        693| 99.56709956709958|
|  Refundable|  Cancelled|   10|         13| 76.92307692307693|
|  No Deposit|  Cancelled|14357|      46970|30.566318926974667|
+------------+-----------+-----+-----------+------------------+



From this query, we conclude that the deposit type NON REFUND is the one with the highest cancellation rate and lowest reservation rate.

And so we are going to investigate more to check what other factors affect the cancellation rate on the NO REFUND type.

In [13]:
from pyspark.sql.functions import col

# Getting the count of each deposit type 
query= df.groupby('deposit_type','is_canceled','customer_type')\
.agg(count("*").alias("count"))\
.sort("deposit_type", ascending=False)

# Changing the 0/1 into categorical values
query = query.withColumn("is_canceled", when(query.is_canceled == "0","Reserved") \
      .when(query.is_canceled == "1","Cancelled") \
      .otherwise(query.is_canceled))

# Adding total count of each type to the query
total_counts = query.groupby('deposit_type').agg(F.sum('count').alias('total_count'))
query = query.join(total_counts, 'deposit_type', 'left')

# Filtering the data to get the reserved data separate from the cancelled
reserved = query.filter((col('is_canceled') == 'Reserved'))
cancelled = query.filter((col('is_canceled') == 'Cancelled'))


# Adding the  reservation rate
reserved=reserved.withColumn("Reservation_Rate",  ((F.col("count"))/ F.col("total_count"))*100 )
reserved = reserved.withColumn("Reservation_Rate", F.format_number(F.col("Reservation_Rate"), 3))
reserved=reserved.sort("Reservation_Rate", ascending=False)

# Adding the  cancellation rate
cancelled=cancelled.withColumn("Cancellation_Rate",  ((F.col("count"))/ F.col("total_count"))*100 )
cancelled = cancelled.withColumn("Cancellation_Rate", F.format_number(F.col("Cancellation_Rate"), 3))
cancelled=cancelled.sort("Cancellation_Rate", ascending=False)


# Filtering the data to get the NON REFUND data 
RnR=reserved.filter((col('deposit_type') == 'Non Refund'))
CnR=cancelled.filter((col('deposit_type') == 'Non Refund'))
RnR.show()
CnR.show()


+------------+-----------+---------------+-----+-----------+----------------+
|deposit_type|is_canceled|  customer_type|count|total_count|Reservation_Rate|
+------------+-----------+---------------+-----+-----------+----------------+
|  Non Refund|   Reserved|Transient-Party|    3|        693|           0.433|
+------------+-----------+---------------+-----+-----------+----------------+

+------------+-----------+---------------+-----+-----------+-----------------+
|deposit_type|is_canceled|  customer_type|count|total_count|Cancellation_Rate|
+------------+-----------+---------------+-----+-----------+-----------------+
|  Non Refund|  Cancelled|      Transient|  603|        693|           87.013|
|  Non Refund|  Cancelled|Transient-Party|   60|        693|            8.658|
|  Non Refund|  Cancelled|       Contract|   27|        693|            3.896|
+------------+-----------+---------------+-----+-----------+-----------------+



From these results we can say that customer type affects the cancellation/reservation rate.

Still we are going to investigate more  other factors that has an effect on  cancellation rate along side the customer type Transient.

In [14]:
from pyspark.sql.functions import col

# Getting the count of each deposit type 
query= df.groupby('deposit_type','is_canceled','customer_type','assigned_room_type')\
.agg(count("*").alias("count"))\
.sort("deposit_type", ascending=False)

# Changing the 0/1 into categorical values
query = query.withColumn("is_canceled", when(query.is_canceled == "0","Reserved") \
      .when(query.is_canceled == "1","Cancelled") \
      .otherwise(query.is_canceled))

# Adding total count of each type to the query
total_counts = query.groupby('deposit_type').agg(F.sum('count').alias('total_count'))
query = query.join(total_counts, 'deposit_type', 'left')

# Filtering the data to get the reserved data separate from the cancelled
reserved = query.filter((col('is_canceled') == 'Reserved'))
cancelled = query.filter((col('is_canceled') == 'Cancelled'))


# Adding the  reservation rate
reserved=reserved.withColumn("Reservation_Rate",  ((F.col("count"))/ F.col("total_count"))*100 )
reserved = reserved.withColumn("Reservation_Rate", F.format_number(F.col("Reservation_Rate"), 3))
reserved=reserved.sort("Reservation_Rate", ascending=False)

# Adding the  cancellation rate
cancelled=cancelled.withColumn("Cancellation_Rate",  ((F.col("count"))/ F.col("total_count"))*100 )
cancelled = cancelled.withColumn("Cancellation_Rate", F.format_number(F.col("Cancellation_Rate"), 3))
cancelled=cancelled.sort("Cancellation_Rate", ascending=False)


# Filtering the data to get the NON REFUND data 
reserved=reserved.filter((col('deposit_type') == 'Non Refund'))
cancelled=cancelled.filter((col('deposit_type') == 'Non Refund'))

#Filtering the data to get the TRANSIENT data
reserved=reserved.filter((col('customer_type') == 'Transient'))
cancelled=cancelled.filter((col('customer_type') == 'Transient'))


reserved.show()
cancelled.show()


+------------+-----------+-------------+------------------+-----+-----------+----------------+
|deposit_type|is_canceled|customer_type|assigned_room_type|count|total_count|Reservation_Rate|
+------------+-----------+-------------+------------------+-----+-----------+----------------+
+------------+-----------+-------------+------------------+-----+-----------+----------------+

+------------+-----------+-------------+------------------+-----+-----------+-----------------+
|deposit_type|is_canceled|customer_type|assigned_room_type|count|total_count|Cancellation_Rate|
+------------+-----------+-------------+------------------+-----+-----------+-----------------+
|  Non Refund|  Cancelled|    Transient|                 A|  591|        693|           85.281|
|  Non Refund|  Cancelled|    Transient|                 B|    8|        693|            1.154|
|  Non Refund|  Cancelled|    Transient|                 E|    2|        693|            0.289|
|  Non Refund|  Cancelled|    Transient|   

From the results we can say that no Transient has completed the reservation.
 

From above queries we can say that both room type and customer type affect the cancellation/reservation rate  
It is suggested that
1) the hotel can offer 5% discount for reservations that of deposite type NON REFUND
2) the hotel offer extra 5% discount over that in 1 for reservations of Customer Type TRANSIENT  
3) the hotel offer extra 5% discount over that in 2 for reservations of Assigned Room Type E & D 

Promotion Three

In [15]:
# Season with most guests
durationOfStay = df.groupby("arrival_date_month", "duration_of_stay")\
.agg(count("*").alias("count"))\
.withColumn("total",  (F.col("count"))* F.col("duration_of_stay") )

season = durationOfStay.groupby("total", "arrival_date_month").count()
season.groupBy("arrival_date_month").sum("total")

# Define the mapping of months to seasons

#Autumn - September, October, November.
#Winter - December, January and February.
#Spring - March, April and May. 
#Summer - June, July and August
from pyspark.sql.functions import when, col, sum

season_mapping = {
    "Autumn": ["September", "October", "November"],
    "Winter": ["December", "January", "February"],
    "Spring": ["March", "April", "May"],
    "Summer": ["June", "July", "August"]
}

# Apply the mapping and calculate the sum for each season
df_transformed = season.withColumn("season",
                               when(col("arrival_date_month").isin(season_mapping["Autumn"]), "Autumn")
                               .when(col("arrival_date_month").isin(season_mapping["Winter"]), "Winter")
                               .when(col("arrival_date_month").isin(season_mapping["Spring"]), "Spring")
                               .when(col("arrival_date_month").isin(season_mapping["Summer"]), "Summer")
                               .otherwise("Unknown")
                               ) \
                   .groupBy("season") \
                   .sum("total")
df_transformed=df_transformed.sort("sum(total)",ascending=False ).show()

+------+----------+
|season|sum(total)|
+------+----------+
|Summer|     53369|
|Spring|     43792|
|Autumn|     30808|
|Winter|     27095|
+------+----------+



In [16]:
from pyspark.sql.functions import col
import pyspark.sql.functions as F

# Getting the data of winter reservations:
query = df.filter((col('arrival_date_month') == 'December') | (col('arrival_date_month') == 'January')|(col('arrival_date_month') == 'February'))

# Getting the cancelled rooms
query=query.filter(col('is_canceled') == 1)

# Getting the most cancelled reserved room
query1=query.groupBy('reserved_room_type').\
  count()\
  .sort("count", ascending=False)
query1.show()

# Based on this query we will filter the data to get room A data
query=query.filter(col('reserved_room_type') == "A")

# Checking agency that made the booking
query2=query.groupBy('agent').\
  count()\
  .sort("count", ascending=False)

# Checking the days of visit
query3=query.groupBy('stays_in_week_nights','stays_in_weekend_nights').\
  count()\
  .sort("count", ascending=False)

# Checking the number of waiting days
query4=query.groupBy('days_in_waiting_list','agent').\
  count()\
  .sort("count", ascending=False)

# Checking the number of waiting days
query5=query.groupBy('distribution_channel').\
  count()\
  .sort("count", ascending=False)
query5.show()

# Filtering the data to get TA/TO data
query11=query.filter(col('distribution_channel') == "TA/TO")

query6=query11.groupBy('country').\
  count()\
  .sort("count", ascending=False)
query6.show()


+------------------+-----+
|reserved_room_type|count|
+------------------+-----+
|                 A| 1807|
|                 D|  487|
|                 F|  109|
|                 B|   84|
|                 E|   37|
|                 G|   12|
+------------------+-----+

+--------------------+-----+
|distribution_channel|count|
+--------------------+-----+
|               TA/TO| 1723|
|              Direct|   70|
|                 GDS|    9|
|           Corporate|    5|
+--------------------+-----+

+-------+-----+
|country|count|
+-------+-----+
|    PRT|  463|
|    ESP|  177|
|    FRA|  175|
|    ITA|  149|
|    BRA|  113|
|    GBR|  103|
|    CHN|   80|
|    DEU|   70|
|    AGO|   49|
|    CHE|   31|
|    NLD|   29|
|    BEL|   25|
|    USA|   22|
|    IRL|   22|
|    ROU|   19|
|    RUS|   18|
|    TUR|   15|
|    AUS|   12|
|    LUX|   11|
|    MAR|    9|
+-------+-----+
only showing top 20 rows



23/06/09 14:07:25 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


From the above queries we can say that most cancellations are done during winter.

We then investigated the factors that affect the cancellations starting be the room type which proved to be A. Then we continued expoloring other factors and one that had an effect on the room type and winter was the distribution channel. Finally we found that most cancellations comes from Portugal.

It is suggested that
1) the hotel can offer 5% discount for reservations that of Room Type  A
2) the hotel offer extra 5% discount over that in 1 for reservations of Distribution Channel TA/TO
3) the hotel offer extra 5% discount over that in 2 for reservations coming from Portugal